In [11]:
!pip install vizdoom

In [11]:
# !cd github & git clone https://github.com/mwydmuch/ViZDoom

The system cannot find the path specified.
Cloning into 'ViZDoom'...
Updating files:   8% (174/2062)
Updating files:   9% (186/2062)
Updating files:  10% (207/2062)
Updating files:  11% (227/2062)
Updating files:  12% (248/2062)
Updating files:  13% (269/2062)
Updating files:  14% (289/2062)
Updating files:  15% (310/2062)
Updating files:  16% (330/2062)
Updating files:  17% (351/2062)
Updating files:  18% (372/2062)
Updating files:  19% (392/2062)
Updating files:  20% (413/2062)
Updating files:  21% (434/2062)
Updating files:  22% (454/2062)
Updating files:  23% (475/2062)
Updating files:  24% (495/2062)
Updating files:  25% (516/2062)
Updating files:  26% (537/2062)
Updating files:  27% (557/2062)
Updating files:  28% (578/2062)
Updating files:  29% (598/2062)
Updating files:  30% (619/2062)
Updating files:  31% (640/2062)
Updating files:  32% (660/2062)
Updating files:  33% (681/2062)
Updating files:  34% (702/2062)
Updating files:  35% (722/2062)
Updating files:  36% (743/2062)
Upd

In [87]:

# Import vizdoom for game env
from vizdoom import * 
# Import random for action sampling
import random
# Import time for sleeping
import time 
# Import numpy for identity matrix
import numpy as np

In [88]:
# Setup game
game = DoomGame()
game.load_config(r'C:\Users\indug\doom\ViZDoom\scenarios\deadly_corridor_s5.cfg')
game.init()

In [89]:
# This is the set of actions we can take in the environment
actions = np.identity(7, dtype=np.uint8)

In [90]:
state = game.get_state()

In [91]:
state.game_variables


array([100.,   0.,   0.,  -1.])

In [92]:
# Loop through episodes 
episodes = 10 
for episode in range(episodes): 
    # Create a new episode or game 
    game.new_episode()
    # Check the game isn't done 
    while not game.is_episode_finished(): 
        # Get the game state 
        state = game.get_state()
        # Get the game image 
        img = state.screen_buffer
        # Get the game variables - ammo
        info = state.game_variables
        # Take an action
        reward = game.make_action(random.choice(actions),4)
        # Print rewward 
        print('reward:', reward) 
        time.sleep(0.02)
    print('Result:', game.get_total_reward())
    time.sleep(2)

reward: 0.0
reward: 0.78125
reward: 2.4580535888671875
reward: 1.657928466796875
reward: -16.271209716796875
reward: -4.572509765625
reward: 0.0
reward: 7.0593719482421875
reward: 8.241668701171875
reward: 5.0597381591796875
reward: 2.5767822265625
reward: 0.4629974365234375
reward: -0.0986785888671875
reward: -7.11376953125
reward: -8.534164428710938
reward: -5.7565460205078125
reward: -1.91925048828125
reward: 2.0606842041015625
reward: 2.45257568359375
reward: 4.3834381103515625
reward: -97.61143493652344
Result: -104.68307495117188
reward: 0.0
reward: -0.78125
reward: -2.458099365234375
reward: -1.6581268310546875
reward: 5.9416046142578125
reward: 7.71514892578125
reward: 5.203887939453125
reward: 3.509979248046875
reward: 2.3674163818359375
reward: 8.6796875
reward: 9.574005126953125
reward: 11.0538330078125
reward: 4.402496337890625
reward: 1.2091522216796875
reward: -6.597198486328125
reward: -8.16961669921875
reward: -5.51068115234375
reward: -3.717132568359375
reward: -2.5073

In [93]:
game.close()

In [138]:
!pip install gym

In [94]:
# Import environment base class from OpenAI Gym
from gym import Env
# Import gym spaces 
from gym.spaces import Discrete, Box
# Import opencv 
import cv2

In [11]:
# game.get_state().screen_buffer.shape

In [97]:
# Create Vizdoom OpenAI Gym Environment
class VizDoomGym(Env): 
    # Function that is called when we start the env
    def __init__(self, render=False, config= r'C:\Users\indug\doom\ViZDoom\scenarios\deadly_corridor_s5.cfg'): 
        # Inherit from Env
        super().__init__()
        # Setup the game 
        self.game = DoomGame()
        self.game.load_config(config)
        
        # Render frame logic
        if render == False: 
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        # Start the game 
        self.game.init()
        
        # Create the action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8) 
        self.action_space = Discrete(7)
        
        # Game variables: HEALTH DAMAGE_TAKEN HITCOUNT SELECTED_WEAPON_AMMO
        self.damage_taken = 0
        self.hitcount = 0
        self.ammo = 52 ## CHANGED
        
        
    # This is how we take a step in the environment
    def step(self, action):
        # Specify action and take step 
        actions = np.identity(7)
        movement_reward = self.game.make_action(actions[action], 4) 
        
        reward = 0 
        # Get all the other stuff we need to retun 
        if self.game.get_state(): 
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            
            # Reward shaping
            game_variables = self.game.get_state().game_variables
            health, damage_taken, hitcount, ammo = game_variables
            
            # Calculate reward deltas
            damage_taken_delta = -damage_taken + self.damage_taken
            self.damage_taken = damage_taken
            hitcount_delta = hitcount - self.hitcount
            self.hitcount = hitcount
            ammo_delta = ammo - self.ammo
            self.ammo = ammo
            
            reward = movement_reward + damage_taken_delta*10 + hitcount_delta*200  + ammo_delta*5 
            info = ammo
        else: 
            state = np.zeros(self.observation_space.shape)
            info = 0 
        
        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, info 
    
    # Define how to render the game or environment 
    def render(): 
        pass
    
    # What happens when we start a new game 
    def reset(self): 
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    # Grayscale the game frame and resize it 
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state
    
    # Call to close down the game
    def close(self): 
        self.game.close()

In [98]:
env = VizDoomGym(render=True)

In [99]:
env.step(3)

(array([[[32],
         [33],
         [25],
         ...,
         [27],
         [23],
         [24]],
 
        [[27],
         [33],
         [23],
         ...,
         [24],
         [24],
         [24]],
 
        [[20],
         [35],
         [23],
         ...,
         [24],
         [24],
         [24]],
 
        ...,
 
        [[75],
         [63],
         [62],
         ...,
         [44],
         [71],
         [60]],
 
        [[15],
         [48],
         [47],
         ...,
         [49],
         [69],
         [47]],
 
        [[22],
         [14],
         [26],
         ...,
         [57],
         [37],
         [39]]], dtype=uint8),
 0.0,
 False,
 {'info': 52.0})

In [100]:
state = env.reset()

In [101]:
env.reset()

array([[[32],
        [33],
        [25],
        ...,
        [27],
        [23],
        [24]],

       [[27],
        [33],
        [23],
        ...,
        [24],
        [24],
        [24]],

       [[20],
        [35],
        [23],
        ...,
        [24],
        [24],
        [24]],

       ...,

       [[75],
        [63],
        [62],
        ...,
        [44],
        [71],
        [60]],

       [[15],
        [48],
        [47],
        ...,
        [49],
        [69],
        [47]],

       [[22],
        [14],
        [26],
        ...,
        [57],
        [37],
        [39]]], dtype=uint8)

In [102]:
env.close()

In [105]:
# Import Environment checker
from stable_baselines3.common import env_checker

In [148]:
# env_checker.check_env(env)

In [149]:

# !pip install matplotlib

In [150]:
# from matplotlib import pyplot as plt

In [151]:

# plt.imshow(cv2.cvtColor(state, cv2.COLOR_BGR2RGB))

In [17]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [45]:
!pip install stable-baselines3[extra]


In [106]:
# Import os for file nav
import os 
# Import callback class from sb3
from stable_baselines3.common.callbacks import BaseCallback

In [107]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [108]:
CHECKPOINT_DIR = './train/train_corridor'
LOG_DIR = './logs/log_corridor'

In [109]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [110]:

# import ppo for training
from stable_baselines3 import PPO

In [111]:
# Non rendered environment
env = VizDoomGym(config = r'C:\Users\indug\doom\ViZDoom\scenarios\deadly_corridor_s5.cfg')

In [113]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.0001, n_steps=2048)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


C:\Users\indug\anaconda3\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [114]:
model.learn(total_timesteps=100000, callback=callback)

Logging to ./logs/log_corridor\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32       |
|    ep_rew_mean     | 9.04     |
| time/              |          |
|    fps             | 13       |
|    iterations      | 1        |
|    time_elapsed    | 148      |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 39.2        |
|    ep_rew_mean          | 17.5        |
| time/                   |             |
|    fps                  | 21          |
|    iterations           | 2           |
|    time_elapsed         | 186         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.025462393 |
|    clip_fraction        | 0.387       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.92       |
|    explained_variance   | -2.86e-

In [115]:
# Import eval policy to test agent
from stable_baselines3.common.evaluation import evaluate_policy

In [116]:

# Reload model from disc
model = PPO.load('./train/train_corridor/best_model_90000')

In [117]:

# Create rendered environment
env = VizDoomGym(render=True)

In [122]:
# Evaluate mean reward for 10 games
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=4)

In [123]:

mean_reward

-3.4269256591796875

In [124]:

for episode in range(5): 
    obs = env.reset()
    done = False
    total_reward = 0
    while not done: 
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        time.sleep(0.80)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(episode, total_reward))
    time.sleep(2)

Total Reward for episode 0 is 146.13624572753906
Total Reward for episode 1 is 217.1281280517578
Total Reward for episode 2 is -255.98895263671875
Total Reward for episode 3 is -33.233673095703125
Total Reward for episode 4 is -65.98152160644531


In [125]:
env.close()

In [205]:
env = VizDoomGym(render=True)

In [214]:
env.step(1)

(array([[[ 6],
         [10],
         [ 9],
         ...,
         [ 6],
         [11],
         [ 4]],
 
        [[ 9],
         [ 9],
         [ 6],
         ...,
         [ 6],
         [ 6],
         [ 4]],
 
        [[10],
         [ 7],
         [ 7],
         ...,
         [ 6],
         [ 6],
         [ 9]],
 
        ...,
 
        [[75],
         [63],
         [62],
         ...,
         [44],
         [71],
         [60]],
 
        [[15],
         [48],
         [47],
         ...,
         [49],
         [69],
         [47]],
 
        [[22],
         [14],
         [26],
         ...,
         [57],
         [37],
         [39]]], dtype=uint8),
 0.0,
 False,
 {'info': 26.0})

In [204]:
env.close()